In [132]:
import os
import polars as pl
import pandas as pd
import re

In [103]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException
from tqdm import tqdm
import math
import time
import os
import polars as pl
import concurrent.futures

def driversetup():
    options = webdriver.ChromeOptions()
    #run Selenium in headless mode
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    #overcome limited resource problems
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument("lang=en")
    #open Browser in maximized mode
    options.add_argument("start-maximized")
    #disable infobars
    options.add_argument("disable-infobars")
    #disable extension
    options.add_argument("--disable-extensions")
#     options.add_argument("--incognito")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_experimental_option('excludeSwitches', ['enable-logging'])
    driver = webdriver.Chrome(options=options)
    driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined});")
    return driver


In [7]:
cur_dir = os.getcwd()
files = [i for i in os.listdir(cur_dir) if '.csv' in i and 'bid' not in i]
files

['0_2500.csv',
 '10000_12500.csv',
 '20000_22500.csv',
 '2500_5000.csv',
 '30000_32500.csv',
 '32500_35000.csv',
 '35000_37500.csv',
 '40000_42500.csv',
 '42500_45000.csv',
 '45000_47500.csv',
 '47500_50000.csv',
 '5000_7500.csv',
 '7500_10000.csv']

In [9]:
# Extract numeric ranges from filenames
intervals = [tuple(map(int, f[:-4].split('_'))) for f in files]

# Sort intervals based on start values
intervals.sort()

# Function to generate missing chunks in 2500 intervals
def find_missing_chunks(intervals, start_range=0, end_range=50000, chunk_size=2500):
    covered = set()  # Store covered chunks

    # Mark all covered chunks
    for start, end in intervals:
        for i in range(start, end, chunk_size):
            covered.add(i)

    # Find missing chunks
    missing_chunks = []
    for i in range(start_range, end_range, chunk_size):
        if i not in covered:
            missing_chunks.append((i, i + chunk_size))

    return missing_chunks

# Get the missing chunks
missing_chunks = find_missing_chunks(intervals)

# Print missing chunks
print("Missing Chunks:", missing_chunks)

Missing Chunks: [(12500, 15000), (15000, 17500), (17500, 20000), (22500, 25000), (25000, 27500), (27500, 30000), (37500, 40000)]


Local: (12500, 15000)
Worker 1: (15000, 17500)
Worker 2: (17500, 20000)
Worker 3: (22500, 25000)
Worker 4: (25000, 27500)
Worker 5: (27500, 30000)
Worker 6: (37500, 40000)


### Spreading out Chunks

In [88]:
par_dir = os.path.dirname(cur_dir)
naics_dir = os.path.join(par_dir, 'NAICS Processed')
file_loc = os.path.join(naics_dir, '236220_second_proc.csv')
df = pl.read_csv(file_loc)
df.head()


Contract ID,NAICS,Date Signed,Contracting Agency ID,PSC,Region,Business Entity ID,Action Obligation ($),Modification Count,Total Modified Action Obligation ($)
str,i64,str,str,str,str,str,i64,i64,i64
"""GS11P14MAP0340""",236220,"""2014-10-16""","""4740""","""Z2AA""","""Southeast""","""RWDWFG6WGRK9""",644325,1,28426
"""IND0407CT66810""",236220,"""2006-12-21""","""1406""","""Y199""","""West""","""U4K9M66MUHR9""",68450,0,0
"""W912LD06C0036""",236220,"""2006-09-25""","""2100""","""Y111""","""Northeast""","""EATDZJL6JFJ5""",12573650,15,556384
"""GS11P06ZGC0339""",236220,"""2006-10-17""","""4740""","""J039""","""Southeast""","""QKCFMFL4MKT6""",11943,1,1504
"""HHSD200200618928C""",236220,"""2006-08-30""","""7523""","""Y111""","""Midwest""","""TTHNY8N2PQR3""",192500,1,38349


In [89]:
sub_dir = os.path.join(par_dir, 'Sub-Award Data')

In [91]:
new_df = df.with_columns(
    pl.Series("Number of Sub-Award Transactions", ["Empty"] * len(df)),
    pl.Series("Sub-Award Obligation Sum", ["Empty"] * len(df)),
    pl.Series("Percent of Prime Award (Sub-Award)", ["Empty"] * len(df))
)

retain_columns = ['Contract ID',
                  'Contracting Agency ID',
                  'Number of Sub-Award Transactions',
                  'Sub-Award Obligation Sum',
                  'Percent of Prime Award (Sub-Award)']

new_df = new_df.select(retain_columns)
new_df.head()

Contract ID,Contracting Agency ID,Number of Sub-Award Transactions,Sub-Award Obligation Sum,Percent of Prime Award (Sub-Award)
str,str,str,str,str
"""GS11P14MAP0340""","""4740""","""Empty""","""Empty""","""Empty"""
"""IND0407CT66810""","""1406""","""Empty""","""Empty""","""Empty"""
"""W912LD06C0036""","""2100""","""Empty""","""Empty""","""Empty"""
"""GS11P06ZGC0339""","""4740""","""Empty""","""Empty""","""Empty"""
"""HHSD200200618928C""","""7523""","""Empty""","""Empty""","""Empty"""


In [92]:
# Define the schema
schema = {
    "Contract ID": pl.Utf8,
    "Sub-Contractor": pl.Utf8,
    "Obligation": pl.Int32
}

# Create an empty DataFrame with the predefined schema
samp_df = pl.DataFrame(schema=schema)

samp_df.head()

Contract ID,Sub-Contractor,Obligation
str,str,i32


In [93]:
workers = 7
increment = n/workers
for i in range(0,7):
    start_range = int(increment*(i+1) - increment)
    end_range = int(increment*(i+1))
    worker_df = new_df[start_range:end_range]
    save_dir_summary = os.path.join(sub_dir, f'Worker{i}_summary_data.csv')
    save_dir_specific = os.path.join(sub_dir, f'Worker{i}_subaward_data.csv')
    worker_df.write_csv(save_dir_summary)
    samp_df.write_csv(save_dir_specific)

worker_df.shape

(7567, 5)

In [94]:
worker_df.head()

Contract ID,Contracting Agency ID,Number of Sub-Award Transactions,Sub-Award Obligation Sum,Percent of Prime Award (Sub-Award)
str,str,str,str,str
"""W9128F10C0069""","""2100""","""Empty""","""Empty""","""Empty"""
"""FA667507C0008""","""5700""","""Empty""","""Empty""","""Empty"""
"""INP13PC00108""","""1443""","""Empty""","""Empty""","""Empty"""
"""GS09P08NPC0005""","""4740""","""Empty""","""Empty""","""Empty"""
"""W91B4M09C4163""","""2100""","""Empty""","""Empty""","""Empty"""


In [122]:
from selenium.common.exceptions import ElementClickInterceptedException
def driversetup():
    options = webdriver.ChromeOptions()
    #run Selenium in headless mode
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    #overcome limited resource problems
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument("lang=en")
    #open Browser in maximized mode
    options.add_argument("start-maximized")
    #disable infobars
    options.add_argument("disable-infobars")
    #disable extension
    options.add_argument("--disable-extensions")
#     options.add_argument("--incognito")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_experimental_option('excludeSwitches', ['enable-logging'])
    driver = webdriver.Chrome(options=options)
    driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined});")
    return driver

work_num = 0

# dataframe setup
file_name_summary = [i for i in os.listdir(sub_dir) if f'Worker{work_num}' in i and 'summary' in i][0]
file_name_specific = [i for i in os.listdir(sub_dir) if f'Worker{work_num}' in i and 'subaward' in i][0]
file_loc_summary = os.path.join(sub_dir, file_name_summary)
file_loc_specific = os.path.join(sub_dir, file_name_specific)
df = pd.read_csv(file_loc_summary)

# driver setup
driver = driversetup()

# scraping

for index, row in tqdm(df.iterrows(), total=len(df), desc="Processing rows"):
    contract_id = row['Contract ID']
    agency_id = row['Contracting Agency ID']
    agency_id = agency_id if agency_id != '2100' else '9700'
    num_sub = row['Number of Sub-Award Transactions']
    sum_sub = row['Sub-Award Obligation Sum']
    percent_sub = row['Percent of Prime Award (Sub-Award)']
    # define some strings
    nums = ''
    sums = ''
    perc = ''
    # checks
    if num_sub != 'Empty' and num_sub != '':
        nums = num_sub
    if sum_sub != 'Empty' and sum_sub != '':
        sums = sum_sub
    if percent_sub != 'Empty' and percent_sub != '':
        perc = percent_sub
    URL = f'https://www.usaspending.gov/award/CONT_AWD_{contract_id}_{agency_id}_-NONE-_-NONE-'
    try:
        driver.get(URL)
    except:
        continue
    time.sleep(2)
    # click on sub awards button
    try:
        driver.find_element(By.XPATH, "/html/body/div/div/main/div/div[6]/div/div[2]/div[1]/div[3]/div").click()
    except (NoSuchElementException, ElementClickInterceptedException):
        continue
    time.sleep(2)
    # get num sub
    
    if nums == '':
        try:
            nums = driver.find_element(By.XPATH, "/html/body/div/div/main/div/div[6]/div/div[2]/div[2]/div[1]/div/div[1]/div/div/div[2]").text
        except NoSuchElementException:
            try:
                time.sleep(0.5)
                nums = driver.find_element(By.XPATH, "/html/body/div/div/main/div/div[6]/div/div[2]/div[2]/div[1]/div/div[1]/div/div/div[2]").text
            except NoSuchElementException:
                nums = ''
        df.at[index, 'Number of Sub-Award Transactions'] = nums
    # get sum
    if sums == '':
        try:
            if nums == 0:
                sums = 0
            else:
                sums = driver.find_element(By.XPATH, "/html/body/div/div/main/div/div[6]/div/div[2]/div[2]/div[1]/div/div[2]/div/div/div[2]").text
        except NoSuchElementException:
            time.sleep(0.5)
            try:
                sums = driver.find_element(By.XPATH, "/html/body/div/div/main/div/div[6]/div/div[2]/div[2]/div[1]/div/div[2]/div/div/div[2]").text
            except NoSuchElementException:
                sums = ''
        df.at[index, 'Sub-Award Obligation Sum'] = sums
    # get percent
    if perc == '':
        try:
            if nums == 0:
                perc = 0
            else:
                perc = driver.find_element(By.XPATH, "/html/body/div/div/main/div/div[6]/div/div[2]/div[2]/div[1]/div/div[3]/div/div/div[2]").text
        except NoSuchElementException:
            time.sleep(0.5)
            try:
                perc = driver.find_element(By.XPATH, "/html/body/div/div/main/div/div[6]/div/div[2]/div[2]/div[1]/div/div[3]/div/div/div[2]").text
            except NoSuchElementException:
                perc = ''
        df.at[index, 'Percent of Prime Award (Sub-Award)'] = perc
    df.to_csv(file_loc_summary)
df.head()
    

Processing rows: 100%|███████████████████████████████████████████████████████████| 7567/7567 [8:26:03<00:00,  4.01s/it]


,Unnamed: 0,Contract ID,Contracting Agency ID,Number of Sub-Award Transactions,Sub-Award Obligation Sum,Percent of Prime Award (Sub-Award)
0,0,GS11P14MAP0340,4740,0,$0,0%
1,1,IND0407CT66810,1406,0,$0,0%
2,2,W912LD06C0036,2100,0,$0,0%
3,3,GS11P06ZGC0339,4740,0,$0,0%
4,4,HHSD200200618928C,7523,0,$0,0%


### Mopper - FPDS Bids

In [ ]:
# Worker 0: 50000-50420 - Done
# Worker 1: 50420-50840 - Done
# Worker 2: 50840-51260 - Done
# Worker 3: 51260-51680 - Done
# Worker 4: 51680-52100 - Done
# Worker 5: 52100-52520 - Done
# Worker 6: 52520-52970 - Done

In [127]:
### start mopping up or finish the last 2970

special_comp_params = ["Follow On to Competed Action",
                       "Not Available for Competition",
                       "Not Competed"]

cur_dir = os.path.dirname(os.getcwd())
naics_data_dir = os.path.join(cur_dir, 'NAICS Processed')
df = pl.read_csv(os.path.join(naics_data_dir, '236220_second_proc.csv'))
driver = driversetup()

ranges = (52520,52970)
comp_type = []
bids = []

for row in tqdm(df[ranges[0]+len(comp_type):ranges[1]].iter_rows(named=True), total=ranges[1]-ranges[0]-len(comp_type), desc="Processing rows"):
    contract_id = row['Contract ID']
    agency_id = row['Contracting Agency ID']
    agency_id = agency_id if agency_id != '2100' else '9700'
    URL = f'https://www.fpds.gov/ezsearch/jsp/viewLinkController.jsp?agencyID={agency_id}&PIID={contract_id}&modNumber=0&transactionNumber=0&idvAgencyID=&idvPIID=&actionSource=searchScreen&actionCode=&documentVersion=1.0&contractType=AWARD&docType=D'
    driver.get(URL)
    # get extent competed and bids
    try:
        extent = driver.find_element(By.CSS_SELECTOR, "#extentCompeted option:checked").text
    except:
        time.sleep(0.5)
        extent = driver.find_element(By.ID, "competitionInformationForDisplay").get_attribute("value")
    bid_num = driver.find_element(By.ID, "numberOfOffersReceived").get_attribute("value")
    # add to lists
    comp_type.append('Open' if extent not in special_comp_params else 'Restricted')
    bids.append(bid_num)
    
df = df[ranges[0]:ranges[1]]
merged_df = df.with_columns([
    pl.Series('Competition Type', comp_type),
    pl.Series('Bids', bids)
])

save_dir = os.path.join(os.getcwd(), f'{ranges[0]}_{ranges[1]}.csv')
merged_df.write_csv(save_dir)

Processing rows: 100%|██████████████████████████████████████████████████████████████▊| 449/450 [13:34<00:01,  1.81s/it]


In [137]:
files = [
    "0_2500.csv", "10000_12500.csv", "12500_15000.csv", "15000_17500.csv",
    "17500_20000.csv", "20000_22500.csv", "22500_25000.csv", "25000_27500.csv",
    "2500_5000.csv", "27500_30000.csv", "30000_32500.csv", "32500_35000.csv",
    "35000_37500.csv", "37500_40000.csv", "40000_42500.csv", "42500_45000.csv",
    "45000_47500.csv", "47500_50000.csv", "50000_50420.csv", "5000_7500.csv",
    "50420_50840.csv", "50840_51260.csv", "51260_51680.csv", "51680_52100.csv",
    "52100_52520.csv", "52520_52970.csv", "7500_10000.csv"
]

# Sort using the first number in the filename
sorted_files = sorted(files, key=lambda x: int(re.match(r"(\d+)", x).group()))
sorted_files

['0_2500.csv',
 '2500_5000.csv',
 '5000_7500.csv',
 '7500_10000.csv',
 '10000_12500.csv',
 '12500_15000.csv',
 '15000_17500.csv',
 '17500_20000.csv',
 '20000_22500.csv',
 '22500_25000.csv',
 '25000_27500.csv',
 '27500_30000.csv',
 '30000_32500.csv',
 '32500_35000.csv',
 '35000_37500.csv',
 '37500_40000.csv',
 '40000_42500.csv',
 '42500_45000.csv',
 '45000_47500.csv',
 '47500_50000.csv',
 '50000_50420.csv',
 '50420_50840.csv',
 '50840_51260.csv',
 '51260_51680.csv',
 '51680_52100.csv',
 '52100_52520.csv',
 '52520_52970.csv']

In [139]:
# combine all bid and comp type data

data_dir = os.getcwd()
combined_df = pl.DataFrame(schema=merged_df.schema)
cur_dir = os.path.dirname(os.getcwd())
naics_data_dir = os.path.join(cur_dir, 'NAICS Processed')
old_df = pl.read_csv(os.path.join(naics_data_dir, '236220_second_proc.csv'))
old_df = old_df.select(["Contract ID"])

for csv_file in sorted_files:
    df_loc = os.path.join(data_dir,csv_file)
    cur_df = pl.read_csv(df_loc)
    combined_df = combined_df.vstack(cur_df)
    
combined_df.head()
# combined_df = old_df.join(combined_df, on="Contract ID", how="inner")


Contract ID,NAICS,Date Signed,Contracting Agency ID,PSC,Region,Business Entity ID,Action Obligation ($),Modification Count,Total Modified Action Obligation ($),Competition Type,Bids
str,i64,str,str,str,str,str,i64,i64,i64,str,str
"""GS11P14MAP0340""",236220,"""2014-10-16""","""4740""","""Z2AA""","""Southeast""","""RWDWFG6WGRK9""",644325,1,28426,"""Restricted""","""1"""
"""IND0407CT66810""",236220,"""2006-12-21""","""1406""","""Y199""","""West""","""U4K9M66MUHR9""",68450,0,0,"""Restricted""","""1"""
"""W912LD06C0036""",236220,"""2006-09-25""","""2100""","""Y111""","""Northeast""","""EATDZJL6JFJ5""",12573650,15,556384,"""Open""","""5"""
"""GS11P06ZGC0339""",236220,"""2006-10-17""","""4740""","""J039""","""Southeast""","""QKCFMFL4MKT6""",11943,1,1504,"""Restricted""","""1"""
"""HHSD200200618928C""",236220,"""2006-08-30""","""7523""","""Y111""","""Midwest""","""TTHNY8N2PQR3""",192500,1,38349,"""Open""","""1"""


In [155]:
# get all missing competition types - should be one error

combined_df.filter(
    (combined_df['Competition Type'] != 'Restricted') & 
    (combined_df['Competition Type'] != 'Open')
)

# get all missing bids

missing_df = combined_df.filter(
    (combined_df['Bids'] == 'Error') |
    (combined_df['Bids'] == '')
)

print(missing_df.shape)
missing_df.tail()

(6911, 12)


Contract ID,NAICS,Date Signed,Contracting Agency ID,PSC,Region,Business Entity ID,Action Obligation ($),Modification Count,Total Modified Action Obligation ($),Competition Type,Bids
str,i64,str,str,str,str,str,i64,i64,i64,str,str
"""N6945009C6836""",236220,"""2009-09-29""","""1700""","""Z199""","""Southwest""","""J6WGTZD3KZ53""",377578,1,53429,"""Open""",""""""
"""N6945009C3132""",236220,"""2009-09-14""","""1700""","""Z199""","""Southwest""","""SHJ9N1LLBLN1""",619109,2,0,"""Open""",""""""
"""N4008011C3522""",236220,"""2011-09-23""","""1700""","""AD61""","""Southeast""","""MS3TYTM12QX5""",260000,2,0,"""Open""",""""""
"""FA461013C0017""",236220,"""2013-06-17""","""5700""","""Z2JZ""","""Midwest""","""ZMS1ZNDY2XV6""",60893,0,0,"""Open""",""""""
"""N4008006C1049""",236220,"""2006-08-02""","""1700""","""Z199""","""Southeast""","""CEHNA6XZU795""",256297,3,38835,"""Open""",""""""


In [159]:
# split up missing into 7 chunks
# Worker 0: 0 to 986 - Running (0)
# Worker 1: 987 to 1973 - Done
# Worker 2: 1974 to 2960 - Running (2)
# Worker 3: 2961 to 3947 - Running (0)
# Worker 4: 3948 to 4934
# Worker 5: 4935 to 5921 - Runnning (5)
# Worker 6: 5922 to 6911

missing_dir = os.path.join(par_dir, 'Missing Bids')
n = missing_df.shape[0]
workers = 7
increment = n/workers

drop_cols = ['Competition Type', 'Bids']

for i in range(0,7):
    start_range = int(increment*(i+1) - increment)
    end_range = int(increment*(i+1))
    end_range = end_range + 1 if i == 6 else end_range
    worker_df = missing_df[start_range:end_range]
    worker_df = worker_df.drop(drop_cols)
    save_dir = os.path.join(missing_dir, f'Worker{i}.csv')
    worker_df.write_csv(save_dir)

print(worker_df.shape)
worker_df.tail()

(988, 10)


Contract ID,NAICS,Date Signed,Contracting Agency ID,PSC,Region,Business Entity ID,Action Obligation ($),Modification Count,Total Modified Action Obligation ($)
str,i64,str,str,str,str,str,i64,i64,i64
"""N6945009C6836""",236220,"""2009-09-29""","""1700""","""Z199""","""Southwest""","""J6WGTZD3KZ53""",377578,1,53429
"""N6945009C3132""",236220,"""2009-09-14""","""1700""","""Z199""","""Southwest""","""SHJ9N1LLBLN1""",619109,2,0
"""N4008011C3522""",236220,"""2011-09-23""","""1700""","""AD61""","""Southeast""","""MS3TYTM12QX5""",260000,2,0
"""FA461013C0017""",236220,"""2013-06-17""","""5700""","""Z2JZ""","""Midwest""","""ZMS1ZNDY2XV6""",60893,0,0
"""N4008006C1049""",236220,"""2006-08-02""","""1700""","""Z199""","""Southeast""","""CEHNA6XZU795""",256297,3,38835


In [163]:
### start mopping up or finish the last 2970

special_comp_params = ["Follow On to Competed Action",
                       "Not Available for Competition",
                       "Not Competed"]

worker_num = 0

missing_dir = os.path.join(par_dir, 'Missing Bids')
df = pl.read_csv(os.path.join(missing_dir, f'Worker{worker_num}.csv'))
driver = driversetup()
comp_type = []
bids = []

for row in tqdm(df[len(comp_type):].iter_rows(named=True), total=df.shape[0]-len(comp_type), desc="Processing rows"):
    contract_id = row['Contract ID']
    agency_id = '9700' # many are defense contracts
#     agency_id = row['Contracting Agency ID']
#     agency_id = agency_id if agency_id != '2100' and agency_id != '5700' else '9700'
    URL = f'https://www.fpds.gov/ezsearch/jsp/viewLinkController.jsp?agencyID={agency_id}&PIID={contract_id}&modNumber=0&transactionNumber=0&idvAgencyID=&idvPIID=&actionSource=searchScreen&actionCode=&documentVersion=1.0&contractType=AWARD&docType=D'
    driver.get(URL)
    # get extent competed and bids
    try:
        extent = driver.find_element(By.CSS_SELECTOR, "#extentCompeted option:checked").text
    except:
        time.sleep(0.5)
        extent = driver.find_element(By.ID, "competitionInformationForDisplay").get_attribute("value")
    bid_num = driver.find_element(By.ID, "numberOfOffersReceived").get_attribute("value")
    # add to lists
    comp_type.append('Open' if extent not in special_comp_params else 'Restricted')
    bids.append(bid_num)
    
merged_df = df.with_columns([
    pl.Series('Competition Type', comp_type),
    pl.Series('Bids', bids)
])

save_dir = os.path.join(missing_dir, f'Worker{worker_num}_data.csv')
merged_df.write_csv(save_dir)

Processing rows: 100%|███████████████████████████████████████████████████████████████| 987/987 [24:00<00:00,  1.46s/it]


In [169]:
# merge all missing with missing by stacking to check if any left

missing_merged = pl.DataFrame(schema=merged_df.schema)

for csv_file in os.listdir(missing_dir):
    if 'data' not in csv_file: 
        continue
    df_loc = os.path.join(missing_dir,csv_file)
    cur_df = pl.read_csv(df_loc)
    missing_merged = missing_merged.vstack(cur_df)

second_pass_missing = missing_merged.filter(
    (missing_merged['Bids'] == 'Error') |
    (missing_merged['Bids'] == '')
)

save_dir = os.path.join(missing_dir, f'second_pass_missing_df.csv')
second_pass_missing.write_csv(save_dir)

In [ ]:
### mop up remaining 3818 into four chunks
# 0, 953: Worker 0 (ran by 2)
# 953, 1906: Worker 1
# 1906, 2859: Worker 4
# 2859, 3818: Worker 5

In [199]:
# merge the remaining 3800 

third_pass_missing_merged = pl.DataFrame(schema=merged_df.schema)

for csv_file in os.listdir(missing_dir):
    if 'second_pass_data' not in csv_file: 
        continue
    df_loc = os.path.join(missing_dir,csv_file)
    cur_df = pl.read_csv(df_loc)
    third_pass_missing_merged = third_pass_missing_merged.vstack(cur_df)

third_pass_missing_merged = third_pass_missing_merged.filter(
    (third_pass_missing_merged['Bids'] == 'Error') |
    (third_pass_missing_merged['Bids'] == '')
)

third_pass_missing_merged

Contract ID,NAICS,Date Signed,Contracting Agency ID,PSC,Region,Business Entity ID,Action Obligation ($),Modification Count,Total Modified Action Obligation ($),Competition Type,Bids
str,i64,str,str,str,str,str,i64,i64,i64,str,str
"""W912PP10C0036""",236220,"""2010-09-29""","""2100""","""Y169""","""Southwest""","""MRT5NMVQQDL8""",2637203,28,1023525,"""Open""",""""""
"""FA890317C0021""",236220,"""2017-07-18""","""5700""","""Y1BZ""","""West""","""GB34Z8NBMBJ6""",243129,0,0,"""Open""",""""""
"""AE103320020""",236220,"""2004-03-19""","""1630""","""Y119""","""Southeast""","""ZFWWD7MYZ5D8""",0,1,249830,"""Open""",""""""
"""DOLJ109630140""",236220,"""2010-04-14""","""1605""","""Y119""","""Southeast""","""NLLKV94M2BP3""",939412,4,137457,"""Open""",""""""
"""FA890316C0001""",236220,"""2016-02-05""","""5700""","""Y1BZ""","""West""","""JM61NJRD58C8""",4831349,7,355681,"""Open""",""""""
…,…,…,…,…,…,…,…,…,…,…,…
"""W15QKN23C0028""",236220,"""2023-07-26""","""2100""","""Y1AB""","""Southeast""","""CMF6MLB57BL4""",1210717,1,0,"""Open""",""""""
"""FA308909C0036""",236220,"""2009-09-30""","""5700""","""Z162""","""Southwest""","""EN3RKWNJ1GY7""",2216422,1,96680,"""Open""",""""""
"""DOLJ041A10011""",236220,"""2004-08-26""","""1630""","""Y249""","""Southeast""","""LVWPXDUBLK96""",798690,0,0,"""Open""",""""""


In [212]:
### start mopping up or finish the last 243

special_comp_params = ["Follow On to Competed Action",
                       "Not Available for Competition",
                       "Not Competed"]

missing_dir = os.path.join(par_dir, 'Missing Bids')
df = third_pass_missing_merged
driver = driversetup()
comp_type = []
bids = []

for row in tqdm(df[len(comp_type):].iter_rows(named=True), total=df.shape[0]-len(comp_type), desc="Processing rows"):
    contract_id = row['Contract ID']
    agency_id = row['Contracting Agency ID']
    agency_id = '9700' if agency_id == '2100' or agency_id == '5700' else agency_id
    agency_id = '1605' if agency_id == '1630' else agency_id
    URL = f'https://www.fpds.gov/ezsearch/jsp/viewLinkController.jsp?agencyID={agency_id}&PIID={contract_id}&modNumber=0&transactionNumber=0&idvAgencyID=&idvPIID=&actionSource=searchScreen&actionCode=&documentVersion=1.0&contractType=AWARD&docType=D'
    driver.get(URL)
    # get extent competed and bids
    try:
        extent = driver.find_element(By.CSS_SELECTOR, "#extentCompeted option:checked").text
    except:
        time.sleep(0.5)
        extent = driver.find_element(By.ID, "competitionInformationForDisplay").get_attribute("value")
    bid_num = driver.find_element(By.ID, "numberOfOffersReceived").get_attribute("value")
    # add to lists
    comp_type.append('Open' if extent not in special_comp_params else 'Restricted')
    bids.append(bid_num)
#     print(f'{contract_id},{agency_id}: {bid_num}')
    
merged_df = df.with_columns([
    pl.Series('Competition Type', comp_type),
    pl.Series('Bids', bids)
])

save_dir = os.path.join(missing_dir, f'third_pass_data.csv')
merged_df.write_csv(save_dir)

merged_df.filter(
    (merged_df['Bids'] == 'Error') |
    (merged_df['Bids'] == '')
)


Processing rows: 100%|███████████████████████████████████████████████████████████████| 243/243 [05:53<00:00,  1.46s/it]


Contract ID,NAICS,Date Signed,Contracting Agency ID,PSC,Region,Business Entity ID,Action Obligation ($),Modification Count,Total Modified Action Obligation ($),Competition Type,Bids
str,i64,str,str,str,str,str,i64,i64,i64,str,str
"""W912PP10C0036""",236220,"""2010-09-29""","""2100""","""Y169""","""Southwest""","""MRT5NMVQQDL8""",2637203,28,1023525,"""Open""",""""""
"""FA890317C0021""",236220,"""2017-07-18""","""5700""","""Y1BZ""","""West""","""GB34Z8NBMBJ6""",243129,0,0,"""Open""",""""""
"""DOLJ109630140""",236220,"""2010-04-14""","""1605""","""Y119""","""Southeast""","""NLLKV94M2BP3""",939412,4,137457,"""Open""",""""""
"""FA890316C0001""",236220,"""2016-02-05""","""5700""","""Y1BZ""","""West""","""JM61NJRD58C8""",4831349,7,355681,"""Open""",""""""
"""FA910115C0049""",236220,"""2015-09-29""","""5700""","""Z2EZ""","""Southeast""","""TZ2RM3HAVK37""",397386,2,5408,"""Open""",""""""
…,…,…,…,…,…,…,…,…,…,…,…
"""FA890319C0017""",236220,"""2019-07-12""","""5700""","""Z1GC""","""West""","""JM61NJRD58C8""",11776250,3,49964,"""Open""",""""""
"""HHSI24120040022CC""",236220,"""2004-04-07""","""7520""","""Z149""","""Midwest""","""F37PJXDMQG76""",167572,1,12353,"""Open""",""""""
"""FA890316C0012""",236220,"""2016-07-28""","""5700""","""Y1BZ""","""West""","""JM61NJRD58C8""",6759532,8,8964411,"""Open""",""""""


In [256]:
# merge all bid info

data_dir = os.getcwd()
combined_bids = pl.DataFrame(schema=merged_df.schema)
cur_dir = os.path.dirname(os.getcwd())
naics_data_dir = os.path.join(cur_dir, 'NAICS Processed')
old_df = pl.read_csv(os.path.join(naics_data_dir, '236220_second_proc.csv'))
old_df = old_df.select(["Contract ID"])

# 0 pass
for csv_file in sorted_files:
    df_loc = os.path.join(data_dir,csv_file)
    cur_df = pl.read_csv(df_loc)
    combined_bids = combined_bids.vstack(cur_df)
    
zero_pass_missing = combined_bids.filter(
    (combined_bids['Bids'] == 'Error') |
    (combined_bids['Bids'] == '')
)

combined_bids = combined_bids.filter(
    ~combined_bids['Contract ID'].is_in(zero_pass_missing['Contract ID'])
)

# first pass
missing_bids_dir = os.path.join(par_dir, 'Missing Bids')

first_pass_combined = pl.DataFrame(schema=combined_bids.schema)

for csv_file in os.listdir(missing_bids_dir):
    if 'pass' in csv_file or 'data' not in csv_file:
        continue
    df_loc = os.path.join(missing_bids_dir,csv_file)
    cur_df = pl.read_csv(df_loc)
    first_pass_combined = first_pass_combined.vstack(cur_df)
    
first_pass_missing = first_pass_combined.filter(
    (first_pass_combined['Bids'] == 'Error') |
    (first_pass_combined['Bids'] == '')
)

combined_bids = combined_bids.vstack(first_pass_combined) # combine the original with first pass
combined_bids = combined_bids.filter(
    ~combined_bids['Contract ID'].is_in(first_pass_missing['Contract ID'])
) # remove those missing again
    
# second pass

second_pass_combined = pl.DataFrame(schema=combined_bids.schema)

for csv_file in os.listdir(missing_bids_dir):
    if 'second_pass' not in csv_file or 'Worker' not in csv_file:
        continue
    df_loc = os.path.join(missing_bids_dir,csv_file)
    cur_df = pl.read_csv(df_loc)
    second_pass_combined = second_pass_combined.vstack(cur_df)
    
second_pass_missing = second_pass_combined.filter(
    (second_pass_combined['Bids'] == 'Error') |
    (second_pass_combined['Bids'] == '')
)

combined_bids = combined_bids.vstack(second_pass_combined) # combine the original with second pass
combined_bids = combined_bids.filter(
    ~combined_bids['Contract ID'].is_in(second_pass_missing['Contract ID'])
) # remove those missing again

# third pass

third_pass_combined = pl.read_csv(os.path.join(missing_bids_dir, 'third_pass_data.csv'), schema=combined_bids.schema)

combined_bids = combined_bids.vstack(third_pass_combined) # combine the original with second pass
combined_bids = combined_bids.filter(
    ~combined_bids['Contract ID'].is_in(third_pass_missing['Contract ID'])
) # remove those missing again

combined_bids

Contract ID,NAICS,Date Signed,Contracting Agency ID,PSC,Region,Business Entity ID,Action Obligation ($),Modification Count,Total Modified Action Obligation ($),Competition Type,Bids
str,i64,str,str,str,str,str,i64,i64,i64,str,str
"""GS11P14MAP0340""",236220,"""2014-10-16""","""4740""","""Z2AA""","""Southeast""","""RWDWFG6WGRK9""",644325,1,28426,"""Restricted""","""1"""
"""IND0407CT66810""",236220,"""2006-12-21""","""1406""","""Y199""","""West""","""U4K9M66MUHR9""",68450,0,0,"""Restricted""","""1"""
"""W912LD06C0036""",236220,"""2006-09-25""","""2100""","""Y111""","""Northeast""","""EATDZJL6JFJ5""",12573650,15,556384,"""Open""","""5"""
"""GS11P06ZGC0339""",236220,"""2006-10-17""","""4740""","""J039""","""Southeast""","""QKCFMFL4MKT6""",11943,1,1504,"""Restricted""","""1"""
"""HHSD200200618928C""",236220,"""2006-08-30""","""7523""","""Y111""","""Midwest""","""TTHNY8N2PQR3""",192500,1,38349,"""Open""","""1"""
…,…,…,…,…,…,…,…,…,…,…,…
"""AE125980220""",236220,"""2004-04-02""","""1630""","""Y119""","""West""","""MY4NMAJ7WBV4""",0,2,98700,"""Open""","""5"""
"""DOLETA14C0039""",236220,"""2014-09-30""","""1630""","""Y1CZ""","""Southeast""","""HPXGPF3LRLX6""",293000,2,7689,"""Restricted""","""1"""
"""DOLJ041A10011""",236220,"""2004-08-26""","""1630""","""Y249""","""Southeast""","""LVWPXDUBLK96""",798690,0,0,"""Restricted""","""1"""


In [263]:
data_dir = os.getcwd()
cur_dir = os.path.dirname(os.getcwd())
naics_data_dir = os.path.join(cur_dir, 'NAICS Processed')
old_df = pl.read_csv(os.path.join(naics_data_dir, '236220_second_proc.csv'))
old_df = old_df.select(["Contract ID"])

# first drop all rows not in old_df

old_df = old_df.filter(
    old_df['Contract ID'].is_in(combined_bids['Contract ID'])
)

# then merge

bid_info_dir = os.path.join
comb_old_bids = combined_bids.join(old_df, on="Contract ID", how="inner")
comb_old_bids

Contract ID,NAICS,Date Signed,Contracting Agency ID,PSC,Region,Business Entity ID,Action Obligation ($),Modification Count,Total Modified Action Obligation ($),Competition Type,Bids
str,i64,str,str,str,str,str,i64,i64,i64,str,str
"""GS11P14MAP0340""",236220,"""2014-10-16""","""4740""","""Z2AA""","""Southeast""","""RWDWFG6WGRK9""",644325,1,28426,"""Restricted""","""1"""
"""IND0407CT66810""",236220,"""2006-12-21""","""1406""","""Y199""","""West""","""U4K9M66MUHR9""",68450,0,0,"""Restricted""","""1"""
"""W912LD06C0036""",236220,"""2006-09-25""","""2100""","""Y111""","""Northeast""","""EATDZJL6JFJ5""",12573650,15,556384,"""Open""","""5"""
"""GS11P06ZGC0339""",236220,"""2006-10-17""","""4740""","""J039""","""Southeast""","""QKCFMFL4MKT6""",11943,1,1504,"""Restricted""","""1"""
"""HHSD200200618928C""",236220,"""2006-08-30""","""7523""","""Y111""","""Midwest""","""TTHNY8N2PQR3""",192500,1,38349,"""Open""","""1"""
…,…,…,…,…,…,…,…,…,…,…,…
"""W90U4210C4063""",236220,"""2010-05-05""","""2100""","""Y111""","""Southeast""","""LN9PU5M2YZN5""",271452,1,0,"""Open""","""1"""
"""FA461013C0017""",236220,"""2013-06-17""","""5700""","""Z2JZ""","""Midwest""","""ZMS1ZNDY2XV6""",60893,0,0,"""Restricted""","""1"""
"""697DCK18C00213""",236220,"""2018-05-03""","""6920""","""Y1BC""","""West""","""X9UNMCNK3A64""",347768,2,0,"""Open""","""4"""


In [268]:
save_dir = os.path.join(os.getcwd(), f'combined_bids.csv')
comb_old_bids.write_csv(save_dir)

In [ ]:
# now merge all the comb missing with the total combined df

# combined_df = combined_df.join(combined_df, on="Contract ID", how="inner")

# then we might have to drop some columns depending on what it looks like

### Mop Up Subaward Information

In [183]:
# merge all subaward information so stack e

retain_columns = ['Contract ID',
                  'Contracting Agency ID',
                  'Number of Sub-Award Transactions',
                  'Sub-Award Obligation Sum',
                  'Percent of Prime Award (Sub-Award)']

samp_df = pl.read_csv(os.path.join(sub_dir,'Worker0_summary_data.csv'))
missing_subs_merged = pl.DataFrame(schema=samp_df.schema).select(retain_columns)

for csv_file in os.listdir(sub_dir):
    if 'summary' in csv_file and '3' not in csv_file: 
        df_loc = os.path.join(sub_dir,csv_file)
        cur_df = pl.read_csv(df_loc).select(retain_columns)
        missing_subs_merged = missing_subs_merged.vstack(cur_df)

# filter for any contracts with subcontractors

# missing_subs_merged.filter(
#     (missing_subs_merged['Number of Sub-Award Transactions'] != 'Empty') &
#     (missing_subs_merged['Number of Sub-Award Transactions'] != '') &
#     (missing_subs_merged['Number of Sub-Award Transactions'] != '0')
# )

# filter for any contracts with 0 values

# missing_subs_merged.filter(
#     (missing_subs_merged['Number of Sub-Award Transactions'] == '0')
# )


# filter for any that are missing rows

missing_subs_merged = missing_subs_merged.filter(
    (missing_subs_merged['Number of Sub-Award Transactions'] == 'Empty') |
    (missing_subs_merged['Number of Sub-Award Transactions'] == '') |
    (missing_subs_merged['Sub-Award Obligation Sum'] == '') |
    (missing_subs_merged['Sub-Award Obligation Sum'] == 'Empty') |
    (missing_subs_merged['Percent of Prime Award (Sub-Award)'] == '') |
    (missing_subs_merged['Percent of Prime Award (Sub-Award)'] == 'Empty') 
)

missing_subs_merged


Contract ID,Contracting Agency ID,Number of Sub-Award Transactions,Sub-Award Obligation Sum,Percent of Prime Award (Sub-Award)
str,str,str,str,str
"""GS11P17NWC0047""","""4740""","""Empty""","""Empty""","""Empty"""
"""W91B4K09C5077""","""2100""","""Empty""","""Empty""","""Empty"""
"""GS11P09NWC0016""","""4740""","""Empty""","""Empty""","""Empty"""
"""N6945018C7211""","""1700""","""Empty""","""Empty""","""Empty"""
"""FA480015C0012""","""5700""","""Empty""","""Empty""","""Empty"""
…,…,…,…,…
"""W90U4210C4063""","""2100""","""Empty""","""Empty""","""Empty"""
"""FA461013C0017""","""5700""","""Empty""","""Empty""","""Empty"""
"""697DCK18C00213""","""6920""","""Empty""","""Empty""","""Empty"""


In [191]:
# chunk it up and distribute among 5 workers (not including results of Worker 3 Yet, because Worker 1 and Worker 3 still working)
# 0 4832: Worker 0
# 4832 9664: Worker 2
# 9664 14496: Worker 4
# 14496 19328: Worker 5
# 19328 24163: Worker 6

missing_subs_dir = os.path.join(par_dir, 'Missing Subs')

# Worker 0

worker_df = missing_subs_merged[0:4832]
save_dir = os.path.join(missing_subs_dir, f'Worker0.csv')
worker_df.write_csv(save_dir)

# Worker 2

worker_df = missing_subs_merged[4832:9664]
save_dir = os.path.join(missing_subs_dir, f'Worker2.csv')
worker_df.write_csv(save_dir)

# Worker 4

worker_df = missing_subs_merged[9664:14496]
save_dir = os.path.join(missing_subs_dir, f'Worker4.csv')
worker_df.write_csv(save_dir)

# Worker 5

worker_df = missing_subs_merged[14496:19328]
save_dir = os.path.join(missing_subs_dir, f'Worker5.csv')
worker_df.write_csv(save_dir)

# Worker 6

worker_df = missing_subs_merged[19328:24163]
save_dir = os.path.join(missing_subs_dir, f'Worker6.csv')
worker_df.write_csv(save_dir)



### Run For those with Non-Zero Subaward Information

In [193]:
from selenium.common.exceptions import ElementClickInterceptedException
def driversetup():
    options = webdriver.ChromeOptions()
    #run Selenium in headless mode
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    #overcome limited resource problems
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument("lang=en")
    #open Browser in maximized mode
    options.add_argument("start-maximized")
    #disable infobars
    options.add_argument("disable-infobars")
    #disable extension
    options.add_argument("--disable-extensions")
#     options.add_argument("--incognito")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_experimental_option('excludeSwitches', ['enable-logging'])
    driver = webdriver.Chrome(options=options)
    driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined});")
    return driver

work_num = 0

# dataframe setup
missing_sub_dir = os.path.join(par_dir, 'Missing Subs')
file_loc = os.path.join(missing_sub_dir, f'Worker{work_num}.csv')
df = pd.read_csv(file_loc)

# driver setup
driver = driversetup()

# scraping

for index, row in tqdm(df.iterrows(), total=len(df), desc="Processing rows"):
    contract_id = row['Contract ID']
    agency_id = row['Contracting Agency ID']
    agency_id = agency_id if agency_id != '2100' else '9700'
    num_sub = row['Number of Sub-Award Transactions']
    sum_sub = row['Sub-Award Obligation Sum']
    percent_sub = row['Percent of Prime Award (Sub-Award)']
    # define some strings
    nums = ''
    sums = ''
    perc = ''
    # checks
    if num_sub != 'Empty' and num_sub != '':
        nums = num_sub
    if sum_sub != 'Empty' and sum_sub != '':
        sums = sum_sub
    if percent_sub != 'Empty' and percent_sub != '':
        perc = percent_sub
    URL = f'https://www.usaspending.gov/award/CONT_AWD_{contract_id}_{agency_id}_-NONE-_-NONE-'
    try:
        driver.get(URL)
    except:
        continue
    time.sleep(2)
    # click on sub awards button
    try:
        driver.find_element(By.XPATH, "/html/body/div/div/main/div/div[6]/div/div[2]/div[1]/div[3]/div").click()
    except (NoSuchElementException, ElementClickInterceptedException):
        continue
    time.sleep(2)
    # get num sub
    
    if nums == '':
        try:
            nums = driver.find_element(By.XPATH, "/html/body/div/div/main/div/div[6]/div/div[2]/div[2]/div[1]/div/div[1]/div/div/div[2]").text
        except NoSuchElementException:
            try:
                time.sleep(0.5)
                nums = driver.find_element(By.XPATH, "/html/body/div/div/main/div/div[6]/div/div[2]/div[2]/div[1]/div/div[1]/div/div/div[2]").text
            except NoSuchElementException:
                nums = ''
        df.at[index, 'Number of Sub-Award Transactions'] = nums
    # get sum
    if sums == '':
        try:
            if nums == 0:
                sums = 0
            else:
                sums = driver.find_element(By.XPATH, "/html/body/div/div/main/div/div[6]/div/div[2]/div[2]/div[1]/div/div[2]/div/div/div[2]").text
        except NoSuchElementException:
            time.sleep(0.5)
            try:
                sums = driver.find_element(By.XPATH, "/html/body/div/div/main/div/div[6]/div/div[2]/div[2]/div[1]/div/div[2]/div/div/div[2]").text
            except NoSuchElementException:
                sums = ''
        df.at[index, 'Sub-Award Obligation Sum'] = sums
    # get percent
    if perc == '':
        try:
            if nums == 0:
                perc = 0
            else:
                perc = driver.find_element(By.XPATH, "/html/body/div/div/main/div/div[6]/div/div[2]/div[2]/div[1]/div/div[3]/div/div/div[2]").text
        except NoSuchElementException:
            time.sleep(0.5)
            try:
                perc = driver.find_element(By.XPATH, "/html/body/div/div/main/div/div[6]/div/div[2]/div[2]/div[1]/div/div[3]/div/div/div[2]").text
            except NoSuchElementException:
                perc = ''
        df.at[index, 'Percent of Prime Award (Sub-Award)'] = perc
    df.to_csv( os.path.join(missing_sub_dir, f'Worker{work_num}_data.csv'))
df.head()
    

Processing rows: 100%|███████████████████████████████████████████████████████████| 4832/4832 [5:17:49<00:00,  3.95s/it]


,Contract ID,Contracting Agency ID,Number of Sub-Award Transactions,Sub-Award Obligation Sum,Percent of Prime Award (Sub-Award)
0,GS11P17NWC0047,4740,,,
1,W91B4K09C5077,2100,Empty,Empty,Empty
2,GS11P09NWC0016,4740,0,$0,0%
3,N6945018C7211,1700,Empty,Empty,Empty
4,FA480015C0012,5700,Empty,Empty,Empty


In [269]:
missing_subs_merged

Contract ID,Contracting Agency ID,Number of Sub-Award Transactions,Sub-Award Obligation Sum,Percent of Prime Award (Sub-Award)
str,str,str,str,str
"""GS11P17NWC0047""","""4740""","""Empty""","""Empty""","""Empty"""
"""W91B4K09C5077""","""2100""","""Empty""","""Empty""","""Empty"""
"""GS11P09NWC0016""","""4740""","""Empty""","""Empty""","""Empty"""
"""N6945018C7211""","""1700""","""Empty""","""Empty""","""Empty"""
"""FA480015C0012""","""5700""","""Empty""","""Empty""","""Empty"""
…,…,…,…,…
"""W90U4210C4063""","""2100""","""Empty""","""Empty""","""Empty"""
"""FA461013C0017""","""5700""","""Empty""","""Empty""","""Empty"""
"""697DCK18C00213""","""6920""","""Empty""","""Empty""","""Empty"""


### Merge Subawards

In [270]:
sub_dir

'C:\\Users\\doran\\Documents\\Procurement Anomaly Detection\\Sub-Award Data'

In [284]:
# merge subawards

# merge all subaward information so stack e

retain_columns = ['Contract ID',
                  'Contracting Agency ID',
                  'Number of Sub-Award Transactions',
                  'Sub-Award Obligation Sum',
                  'Percent of Prime Award (Sub-Award)']

samp_df = pl.read_csv(os.path.join(sub_dir,'Worker0_summary_data.csv'))
subs_merged = pl.DataFrame(schema=samp_df.schema).select(retain_columns)

for csv_file in os.listdir(sub_dir):
    if 'summary' in csv_file and '3' not in csv_file: 
        df_loc = os.path.join(sub_dir,csv_file)
        cur_df = pl.read_csv(df_loc).select(retain_columns)
        subs_merged = subs_merged.vstack(cur_df)

missing_subs_merged = subs_merged.filter(
    (subs_merged['Number of Sub-Award Transactions'] == 'Empty') |
    (subs_merged['Number of Sub-Award Transactions'] == '') |
    (subs_merged['Sub-Award Obligation Sum'] == '') |
    (subs_merged['Sub-Award Obligation Sum'] == 'Empty') |
    (subs_merged['Percent of Prime Award (Sub-Award)'] == '') |
    (subs_merged['Percent of Prime Award (Sub-Award)'] == 'Empty') 
)

# missing_subs_merged

subs_merged = subs_merged.filter(
    ~subs_merged['Contract ID'].is_in(missing_subs_merged['Contract ID'])
)

# first pass

missing_subs_dir = os.path.join(par_dir, 'Missing Subs')
first_pass = pl.DataFrame(schema=subs_merged.schema)
for csv_file in os.listdir(missing_subs_dir):
    if 'data' not in csv_file: continue
    df_loc = os.path.join(missing_subs_dir,csv_file)
    cur_df = pl.read_csv(df_loc).select(retain_columns)
    first_pass = first_pass.vstack(cur_df)
    
subs_merged = subs_merged.vstack(first_pass)

# add 3 to the end
three = pl.read_csv(os.path.join(sub_dir, 'Worker3_summary_data.csv')).select(retain_columns)
subs_merged.vstack(three)

Contract ID,Contracting Agency ID,Number of Sub-Award Transactions,Sub-Award Obligation Sum,Percent of Prime Award (Sub-Award)
str,str,str,str,str
"""GS11P14MAP0340""","""4740""","""0""","""$0""","""0%"""
"""IND0407CT66810""","""1406""","""0""","""$0""","""0%"""
"""W912LD06C0036""","""2100""","""0""","""$0""","""0%"""
"""GS11P06ZGC0339""","""4740""","""0""","""$0""","""0%"""
"""HHSD200200618928C""","""7523""","""0""","""$0""","""0%"""
…,…,…,…,…
"""47PK0620C0004""","""4740""","""Empty""","""Empty""","""Empty"""
"""47PH0520C0002""","""4740""",null,null,null
"""V541C0016""","""3600""","""Empty""","""Empty""","""Empty"""


In [285]:
save_dir = os.path.join(sub_dir, 'combined_sub_award.csv')
subs_merged.write_csv(save_dir)